In [1]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch  # Import the torch module
import pandas as pd

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential

import tensorflow as tf
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,accuracy_score,f1_score,precision_score

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential

import tensorflow as tf
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,accuracy_score,f1_score,precision_score
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3,MobileNet

# Step 1: Preprocess and load the WSI data
train_data_directory = '//content/drive/MyDrive/Train_Set_2'  # Directory containing training patches
test_data_directory = '//content/drive/MyDrive/Test'    # Directory containing test patches

img_height, img_width = 299, 299  # InceptionV3 input size
batch_size = 32
num_classes = 3  # Number of classes: cancerous, precancerous, and non-cancerous

# Define Albumentations transformations
train_transform = A.Compose([
    A.Resize(img_height, img_width),
    A.RandomBrightnessContrast(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=45, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.Normalize(),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(img_height, img_width),
    A.Normalize(),
    ToTensorV2()
])

# Create custom data loaders using Albumentations and PyTorch
from torchvision import datasets, transforms

class CustomDataset(datasets.ImageFolder):
    def __init__(self, data_directory, transform=None):
        super().__init__(root=data_directory, transform=transform)


# Create ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load and preprocess the training and test data
train_generator = train_datagen.flow_from_directory(
    train_data_directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    shuffle=False
)



# Load and preprocess the training and test data using Albumentations
train_dataset = CustomDataset(train_data_directory, transform=train_transform)
test_dataset = CustomDataset(test_data_directory, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



# Step 2: Load the InceptionV3 model and modify it for transfer learning
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 1024 units and ReLU activation
x = Dense(1024, activation='relu')(x)

# Add a final softmax layer for multi-class classification
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model and the new layers
model = Model(inputs=base_model.input, outputs=predictions)

# Step 3: Freeze some layers in the base model for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Step 4: Compile and train the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10  # Choose the number of epochs suitable for your dataset
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs
)

# Step 5: Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test accuracy: {accuracy:.2f}")




# Step 5: Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator, steps=None)
print(f"Test accuracy: {accuracy:.2f}")

# Step 6: Compute category-wise accuracy and F1 score
y_true = test_generator.classes
y_pred = model.predict(test_generator, steps=None).argmax(axis=1)

class_names = list(test_generator.class_indices.keys())

# Compute category-wise accuracy
accuracy_per_class = {}
for class_name in class_names:
  indices = np.where(y_true == test_generator.class_indices[class_name])[0]
  accuracy_per_class[class_name] = np.mean(y_true[indices] == y_pred[indices])

print("Category-wise Accuracy:")
for class_name, acc in accuracy_per_class.items():
  print(f"{class_name}: {acc:.2f}")

# Compute category-wise F1 score
f1_per_class = {}
for class_name in class_names:
  indices = np.where(y_true == test_generator.class_indices[class_name])[0]
  f1_per_class[class_name] = f1_score(y_true[indices], y_pred[indices], average='weighted')

print("\nCategory-wise F1 Score:")
for class_name, f1 in f1_per_class.items():
  print(f"{class_name}: {f1:.2f}")

# Compute overall accuracy and F1 score
overall_accuracy = np.mean(y_true == y_pred)
overall_f1 = f1_score(y_true, y_pred, average='weighted')
print("\nOverall Accuracy:", overall_accuracy)
print("Overall F1 Score:", overall_f1)

# Step 7: Check count of images in each category in test data
test_data_counts = {}
for class_name in class_names:
    class_directory = os.path.join(test_data_directory, class_name)
    num_images = len(os.listdir(class_directory))
    test_data_counts[class_name] = num_images

print("\nCount of Images in Each Category in Test Data:")
for class_name, count in test_data_counts.items():
    print(f"{class_name}: {count}")

FileNotFoundError: [WinError 53] The network path was not found: '//content/drive/MyDrive/Train_Set_2'

In [ ]:


import keras

def get_top_predicted_indices(predictions, top_n):
    return np.argsort(-predictions).squeeze()[:top_n]

def make_gradcam_heatmap(
    img_array, model,
    last_conv_layer_name,
    classifier_layer_names,
    top_n,
    class_indices
):
    #1. Create a model that maps the input image to the activations of the last convolution layer - Get last conv layer's output dimensions
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    #2. Create another model, that maps from last convolution layer to the final class predictions - This is the classifier model that calculated the gradient
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    #3. If top N predictions are to be interospected, Get their Imagenet indices else assign the indices given
    if(top_n > 0):
        last_conv_layer_output = last_conv_layer_model(img_array)
        preds = classifier_model(last_conv_layer_output)
        class_indices = get_top_predicted_indices(preds, top_n)
    else:
        top_n = len(class_indices)

    #4. Create an array to store the heatmaps
    heatmaps = []
    #5. Iteratively calculate heatmaps for all classes of interest using GradientTape
    for index in np.arange(top_n):

        #6. Watch the last convolution output during the prediction process to calculate the gradients
        #7. Compute the activations of last conv layer and make the tape to watch
        with tf.GradientTape() as tape:
            # Compute activations of the last conv layer and make the tape watch it
            last_conv_layer_output = last_conv_layer_model(img_array)
            tape.watch(last_conv_layer_output)

            #8. Get the class predictions and the class channel using the class index
            preds = classifier_model(last_conv_layer_output)
            class_channel = preds[:, class_indices[index]]

        #9. Using tape, Get the gradient for the predicted class wrt the output feature map of last conv layer
        grads = tape.gradient(
            class_channel,
            last_conv_layer_output
        )

        #10. Calculate the mean intensity of the gradient over its feature map channel
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        last_conv_layer_output = last_conv_layer_output.numpy()[0]
        pooled_grads = pooled_grads.numpy()

        #11. Multiply each channel in feature map array by weight importance of the channel
        for i in range(pooled_grads.shape[-1]):
            last_conv_layer_output[:, :, i] *= pooled_grads[i]

        #12. The channel-wise mean of the resulting feature map is our heatmap of class activation
        heatmap = np.mean(last_conv_layer_output, axis=-1)

        #13. Normalize the heatmap between [0, 1] for ease of visualization
        heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

        heatmaps.append({
            "class_id": class_indices[index],
            "heatmap": heatmap
        })

    return heatmaps


from PIL import Image
import torch
import torchvision.transforms as transforms

# Load the image from file
image_path = "/content/drive/MyDrive/WSIs/Cancerous/BRACS_1003699.jpg"
image = Image.open(image_path)
new_image = image.resize((299, 299))
imgdata = np.array(new_image)
imgdata = np.expand_dims(imgdata, axis=0)
y_pred = model.predict(imgdata)
y_pred = np.argmax(y_pred, axis=1)
y_pred


import keras.utils as image

def get_img_array(img_path, size):
    img = image.load_img(img_path, target_size=size)
    array = image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

LAST_CONV_LAYER = 'block5_conv3'
CLASSIFIER_LAYER_NAMES = [
            "block5_pool",
            "global_average_pooling2d_2",
            "dense_4",
            "dense_5"
        ]

# Calculate Heatmaps for TOP_N Predictions
heatmaps = make_gradcam_heatmap(
    get_img_array(image_path, (299, 299)),
    model,
    'block5_conv3',
    CLASSIFIER_LAYER_NAMES,
    3,
    None
)

CLASS_DICT = {'0':'Cancerous','1':'Non Cancerous','2':'Pre Cancerous'}
def display_superimposed_heatmaps(heatmaps, image_path, image_id):
    n = len(heatmaps)
    n_rows = (n // 3) + 1 if n % 3 > 0 else n // 3
    plt.rcParams['axes.grid'] = False
    plt.rcParams['xtick.labelsize'] = False
    plt.rcParams['ytick.labelsize'] = False
    plt.rcParams['xtick.top'] = False
    plt.rcParams['xtick.bottom'] = False
    plt.rcParams['ytick.left'] = False
    plt.rcParams['ytick.right'] = False
    plt.rcParams['figure.figsize'] = [30, 15]
    for index in np.arange(n):
        heatmap = heatmaps[index]["heatmap"]
        class_id = heatmaps[index]["class_id"]
        class_name = CLASS_DICT[str(class_id)].split(",")[0].capitalize()
        superimposed_image = superimpose_heatmap(image_path, heatmap)
        plt.subplot(n_rows, 3, index+1)
        plt.title(f"{class_id}, {class_name}", fontsize= 30)
        plt.imshow(superimposed_image)

    plt.show()
display_superimposed_heatmaps(heatmaps, image_path, 1)
